In [1]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [4]:
# make use change the volumes to avaialbe at the chameleon
cinder_client = chi.clients.cinder()
volume = [v for v in cinder_client.volumes.list() if v.name=='block-persist-project45'][0] # Substitute your own net ID

# print(volume.status)
cinder_client.volumes.delete(volume = volume)

(<Response [202]>, None)

In [9]:
context.choose_project()
context.choose_site(default="CHI@TACC")

# run in Chameleon Jupyter environment
os_conn = chi.clients.connection()
token = os_conn.authorize()
storage_url = os_conn.object_store.get_endpoint()

import swiftclient
swift_conn = swiftclient.Connection(preauthurl=storage_url,
                                    preauthtoken=token,
                                    retries=5)
container_name = "object-persist-project45"
while True:
    _, objects = swift_conn.get_container(container_name, full_listing=True)
    if not objects:
        break
    paths = "\n".join(f"{container_name}/{obj['name']}" for obj in objects)
    swift_conn.post_account(
        headers={"Content-Type": "text/plain"},
        data=paths,
        query_string="bulk-delete"
    )
swift_conn.delete_container(container_name)
print("Container deleted.")

ClientException: Container GET failed: https://chi.tacc.chameleoncloud.org:7480/swift/v1/AUTH_d3c6e101843a4ba79e665ebf59b521a2/object-persist-project45?format=json 404 Not Found  [first 60 chars of response] b'{"Code":"NoSuchBucket","Message":"","BucketName":"object-per' (txn: tx0000036730f4e9e185356-00681c0379-ce6c0deb-default)